In [2]:
#Set Up
import requests
import json
import numpy as np
import pandas as pd
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries

In [3]:
weather_data = pd.read_csv("../WeatherPy/weather_data")
gkey="***"
gmaps.configure(api_key=gkey)

# Heat Map

In [361]:
coords = weather_data[['Latitude', 'Longitudes', 'Humidity']]
coords

,Latitude,Longitudes,Humidity
0,78.22,15.64,85
1,-54.80,-68.30,57
2,77.48,-69.36,86
3,42.10,-79.24,55
4,18.45,-78.17,74
...,...,...,...
815,58.13,68.59,74
816,-7.80,-35.59,84
817,53.70,53.67,75
818,53.75,62.07,64


In [362]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    coords[['Latitude', 'Longitudes']], weights=coords['Humidity'],
    max_intensity=30, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Closest Hotels

In [299]:
slim_data = weather_data[(weather_data.Temperature <80) & (weather_data.Temperature < 70) & (weather_data.Windspeed < 10) & (weather_data.Cloudiness < 1)]
slim_data['LatLong'] = slim_data['Latitude'].astype(str) + ", " + slim_data['Longitudes'].astype(str)

/Users/johnpatten/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [328]:
hotels = []
vicinities = []
global_codes = []
target_search = "hotel"
radius = 5000

params = {
    "location": dc_coords,
    "keyword": target_search,
    "radius": radius,
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [329]:
for index, row in slim_data.iterrows():
    dc_coords = row['LatLong']
    params['location'] = dc_coords
    places_data = requests.get(base_url, params).json()
    try:
        hotel_name = places_data["results"][0]["name"]
        hotels.append(hotel_name)
        vicinity = places_data["results"][0]["vicinity"]
        vicinities.append(vicinity)
        global_code = places_data["results"][0]["plus_code"]["compound_code"]
        global_codes.append(global_code)
        print(hotel_name)
        print(vicinity)
        print(global_code)
    except (KeyError, IndexError):
        hotel_name = "NaN"
        hotels.append(hotel_name)
        vicinity = "NaN"
        vicinities.append(vicinity)
        global_code = "NaN"
        global_codes.append(global_code)
        print("Error")    

The Halyards Hotel
1 Albany Road, Port Alfred
CV4V+77 Port Alfred, South Africa
Millbury Guesthouse
27 Mill Park Rd, Mill Park, Port Elizabeth
2HQR+92 Port Elizabeth, South Africa
Blue Lagoon Hotel
21 Blue Bend Pl, Blue Bend, East London
2X82+JC East London, South Africa
Bertelli Chuí Hotel
Rodovia BR-471 s/n - Centro, Chuí
8H63+P9 Centro, Chui - State of Rio Grande do Sul, Brazil
Commissariat De Police Tsiroanomandidy
Tsiroanomandidy
63J2+RJ Tsiroanomandidy, Madagascar
Apartamenty Porto
Nadbrzeżna 5A, Giżycko
2QM7+V3 Gizycko, Poland
Princesse Tsiribihina
Miandrivazo
FC4X+FX Miandrivazo, Madagascar
Hotel Klaksvík
Klaksvík
6CH4+84 Klaksvík, Faroe Islands
Clarion Hotel Grand Östersund
Prästgatan 16, Östersund
5JHP+WF Östersund, Sweden
Hotel Villa Victoria de Tigre
DAX, Av. Liniers 566, Tigre
HCP5+CH Tigre, Buenos Aires Province, Argentina
Odar kanawal
Autoroute Akjoujt, Atar
FWRP+8R Atar, Mauritania
Error
Hotel ELBLĄG
Stary Rynek 54-59, Elbląg
595W+W6 Elblag, Poland
Error
Error
Гостинниц

In [335]:
slim_data['hotels'] = hotels
slim_data['City, Country'] = global_codes
slimmed_data = slim_data[slim_data['hotels'] != "NaN"]
slimmed_data = slim_data[slim_data['City, Country'] != "NaN"]
locations = slimmed_data[['Latitude', 'Longitudes']]
slimmed_data['City, Country'] = slimmed_data['City, Country'].str[8:]
slimmed_data.head()

/Users/johnpatten/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/johnpatten/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/johnpatten/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Unnamed: 0,Names,City Number,Temperature,Hemisphere,Latitude,Longitudes,Cloudiness,Humidity,Windspeed,LatLong,hotels,"City, Country"
12,12,Port Alfred,13,16.67,Southern,-33.59,26.89,0,84,2.24,"-33.59, 26.89",The Halyards Hotel,"Port Alfred, South Africa"
14,14,Port Elizabeth,15,15.00,Southern,-33.92,25.57,0,100,1.00,"-33.92, 25.57",Millbury Guesthouse,"Port Elizabeth, South Africa"
24,24,East London,25,20.05,Southern,-33.02,27.91,0,61,6.32,"-33.02, 27.91",Blue Lagoon Hotel,"East London, South Africa"
35,35,Chui,36,17.43,Southern,-33.70,-53.46,0,87,4.16,"-33.7, -53.46",Bertelli Chuí Hotel,"Centro, Chui - State of Rio Grande do Sul, Brazil"
37,37,Tsiroanomandidy,38,16.61,Southern,-18.77,46.03,0,68,1.23,"-18.77, 46.03",Commissariat De Police Tsiroanomandidy,"Tsiroanomandidy, Madagascar"


In [336]:
hotel_slim = slimmed_data[['hotels', 'City, Country', 'LatLong']]
hotel_slim.head()

,hotels,"City, Country",LatLong
12,The Halyards Hotel,"Port Alfred, South Africa","-33.59, 26.89"
14,Millbury Guesthouse,"Port Elizabeth, South Africa","-33.92, 25.57"
24,Blue Lagoon Hotel,"East London, South Africa","-33.02, 27.91"
35,Bertelli Chuí Hotel,"Centro, Chui - State of Rio Grande do Sul, Brazil","-33.7, -53.46"
37,Commissariat De Police Tsiroanomandidy,"Tsiroanomandidy, Madagascar","-18.77, 46.03"


In [337]:
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotels}</dd>
<dt>Location</dt><dd>{City, Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_slim.iterrows()]

In [338]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
heatmap_layer = gmaps.heatmap_layer(
    coords[['Latitude', 'Longitudes']], weights=coords['Humidity'],
    max_intensity=30, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))